In [16]:
import os
os.environ["env"] = "dev"
os.environ["AWS_PROFILE"] = "riyaz"
import json
import subprocess
import uuid
import IPython.display as ipd
from playlist_copilot.utils.smp_utils import get_track_info
from recommendation_wrapper import order_by_similarity

In [17]:
# Function to read .m3u file and extract MP3 file paths
def read_m3u(file_path):
    mp3_file_paths = []

    with open(file_path, 'r') as file:
        for line in file:
            # Skip empty lines and lines that start with '#'
            if line.strip() and not line.startswith('#'):
                mp3_file_paths.append(line.strip())

    return mp3_file_paths

# Function to display the songs selected and create a json using which audio can be played
def print_playlist_songnames(file_paths, filename):
    json_for_audio_playback = []
    n=1
    for path in file_paths:
        track_id = os.path.splitext(os.path.basename(path))[0]
        track_info = get_track_info(f'spotify:track:{track_id}')
        track_json = {}
        artist = track_info['artist']
        title = track_info['title']
        album = track_info['album']
        track_json['sequence'] = n
        track_json['title'] = title
        track_json['artist'] = artist
        track_json['album'] = album
        track_json['audio_filepath'] = f'http://192.168.0.156:8000/{track_id}.mp3'
        print(f"{artist} | {title} | {album}")
        json_for_audio_playback.append(track_json)
        n+=1
    with open(f'{filename}.json', 'w') as file:
        json.dump(json_for_audio_playback, file, indent=4)
    

In [18]:
# Pop
# Taylor Swift | The Outside | Taylor Swift
test_1 = '/datasets/universal-music-group-dataset/audio/1U6riPmEZzICc3NVAlxihh.mp3'

# Pop
# Ellie Goulding | Human | Bright Lights
test_2 = '/datasets/universal-music-group-dataset/audio/7l6cyr5rzGRbsTLONoPOHR.mp3'

# Pop
# Busted | Year 3000 | Busted
test_3 = '/datasets/universal-music-group-dataset/audio/0zjB7ewBWnMgQDMQpFzt92.mp3'

# Hip Hop / Rap
# Nicki Minaj | Regret In Your Tears | Regret In Your Tears
test_4 = '/datasets/universal-music-group-dataset/audio/5x8W6iNXDCQrHp1DF5j4UW.mp3'

# Hop Hop / Rap
# 50 Cent | Straight To The Bank | Curtis
test_5 = '/datasets/universal-music-group-dataset/audio/3tFI8AEosXedl4jEQwwwSg.mp3'

# Rock
# Guns N' Roses | Coma | Use Your Illusion I
test_6 = '/datasets/universal-music-group-dataset/audio/34Lo0oc0QDJ1wlepfnWHQI.mp3'

# Orchestral
# George Frideric Handel | Water Music Suite No.1 in F, HWV 348: 9. (Andante) | Handel: Water Music
test_7 = '/datasets/universal-music-group-dataset/audio/0HeK9VsxEJZ8QOQozHJSJY.mp3'

# Jazz
# Quincy Jones | Theme From "The Anderson Tapes" | Smackwater Jack
test_8 = '/datasets/universal-music-group-dataset/audio/1qupALx3nHbzBCS3WIEQZ6.mp3'

# Pop
# Justin Bieber | We Are | Purpose
test_9 = '/datasets/universal-music-group-dataset/audio/7isWH8NT7djPZw5HWlbiun.mp3'

# Pop
# Nelly Furtado | Showtime | Loose
test_10 = '/datasets/universal-music-group-dataset/audio/4cfj6rhDqIgOa9upIYBfl2.mp3'


In [19]:
# USE CASE 1:= Starting from a single seed song

# Choose parameters and song
input_file = test_1

# Additional parameters, if needed
num_songs = '30'  # Number of songs in the playlist
noise = '0'   # Amount of noise in the playlist (default 0)
lookback = '5' # Amount of lookback in the playlist (default 3)

# Path to Python script
script = 'Deej-A.I.py'

# Arguments for the script
mp3tovec = 'mp3tovec'
output_file = str(uuid.uuid4()) + '.m3u'

# Construct the command as a list
command = ['python', script, 'UMGPickles', mp3tovec, '--playlist', output_file,
           '--nsongs', num_songs, '--noise', noise, '--lookback', lookback,
           '--inputsong', input_file]

# Execute the command
subprocess.run(command)

m3u_file_path = output_file

# Read the .m3u file and get the MP3 file paths
mp3_file_paths = read_m3u(m3u_file_path)

rec1_ids = []
for path in mp3_file_paths:
    rec1_ids.append(os.path.splitext(os.path.basename(path))[0])

# print_playlist_songnames(mp3_file_paths, os.path.splitext(m3u_file_path)[0])

filtered_ids = order_by_similarity(rec1_ids, 10)
filtered_m3u_filepath = f"filtered_{output_file}"
filtered_paths = [input_file]
for id in filtered_ids:
    line = f'/datasets/universal-music-group-dataset/audio/{id}.mp3'
    filtered_paths.append(line)

print('\n ---- RECOMMENDATIONS ---- \n')
print_playlist_songnames(filtered_paths, os.path.splitext(filtered_m3u_filepath)[0])
print('\n\n\n')
print(f'Outfile: {filtered_m3u_filepath}')

40048 MP3s
Outfile playlist: aa1f2fdb-36a8-4f68-ac13-e7949d0eefca.m3u
Input song selected: /datasets/universal-music-group-dataset/audio/1U6riPmEZzICc3NVAlxihh.mp3
Requested 30 songs

 ---- RECOMMENDATIONS ---- 

Taylor Swift | The Outside | Taylor Swift
Josh Kelley | Don't You Go | Georgia Clay
Reba McEntire | When You Love Someone Like That | Reba Duets
Rascal Flatts | Words I Couldn't Say | Me And My Gang
Rascal Flatts | I Feel Bad | Me And My Gang
Trisha Yearwood | I Would've Loved You Anyway | Inside Out
Sugarland | One Blue Sky | Enjoy The Ride
Gary Allan | Her Man | Used Heart For Sale
Eros Ramazzotti | Il Viaggio | Perfetto
Rod Stewart | Can We Stay Home Tonight? | Another Country
Jessie J | Harder We Fall | Alive




Outfile: filtered_aa1f2fdb-36a8-4f68-ac13-e7949d0eefca.m3u
